# Gauss Seidel with and without Atkin Acceleration

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def forward_elimination(_A,_b):
    n =  np.shape(_A)[0]
    A = np.copy(_A)
    b = np.copy(_b)
    for k in range(0,n-1):
        for i in range(k+1,n):
            m = A[i,k]/A[k,k]
            for j in range(k,n):
                #print(i, j, A[i,j], A[k,j])
                A[i,j] = A[i,j] - m*A[k,j]
            b[i] = b[i] - m*b[k]
    #print(A,b)
    return A, b

In [3]:
def back_substitution(A,b):
    n = np.shape(A)[0]
    x = np.zeros(n)
    x[n-1] = b[n-1] / A[n-1,n-1]
    for i in range(n-2,-1,-1):
        #print(i)
        _sum = 0
        for j in range(n-1,i,-1):
            #print("j:", j)
            _sum += A[i,j] * x[j]
        x[i] = (b[i] - _sum) / A[i,i]
    
    #print(x)
    return x

In [5]:
A = np.array([[-2.2,1,1], [0.8,-2.2,1], [1,0.9,-2.1]])
print("A:\n" , A)
M = np.array([[-2.2,0,0], [0.8,-2.2,0], [1,0.9,-2.1]])
print("M:\n", M)
N = np.array([[0,-1,-1], [0,0,-1], [0,0,0]])
print("N:\n", N)
y = np.array([.2,.4,.2])
print("y:\n", y)
x0 = np.zeros((3))

A:
 [[-2.2  1.   1. ]
 [ 0.8 -2.2  1. ]
 [ 1.   0.9 -2.1]]
M:
 [[-2.2  0.   0. ]
 [ 0.8 -2.2  0. ]
 [ 1.   0.9 -2.1]]
N:
 [[ 0 -1 -1]
 [ 0  0 -1]
 [ 0  0  0]]
y:
 [0.2 0.4 0.2]


In [6]:
def gs_step(_M, _N, xk, _y):
    C, b = forward_elimination(_M, np.matmul(_N, xk) + _y)
    return back_substitution(C,b)

def gauss_seidel(M, N, x0, y, iterations: int):
    x = np.copy(x0)
    history = []
    history.append(x0)
    for i in range(iterations):
        x = gs_step(M,N,x,y)
        #print(x)
        history.append(x)
    return x, history

def gauss_seidel_check(M, N, x0, y, accuracy: float, exact_x):
    x = np.copy(x0)
    itr = 0
    while (np.max(x-exact_x) > accuracy):
        x = gs_step(M,N,x,y)
        itr += 1
    return x, itr

def aitken_accel(x_p, x_c, x_n):
    d = np.shape(x_p)[0]
    _x = np.zeros(d)
    for i in range(d):
        if (x_n[i] - 2*x_c[i] + x_p[i]) == 0:
            _x[i] = 0
        else:
            _x[i] = (x_n[i] - x_c[i])**2 / (x_n[i] - 2*x_c[i] + x_p[i])
    return _x
   
def aitken(M, N, x0, y, iterations):
    x_prev = np.copy(x0)
    x_curr = np.copy(x0)
    x_next = np.copy(x0)
    for i in range(iterations):
        x = gs_step(M,N,x_next,y)
        #print("gs_step:" , x)
        #print("aitken_accel:" , aitken_accel(x_curr, x_next,x))
        x = x - aitken_accel(x_curr, x_next,x)
        x_prev = x_curr
        x_curr = x_next
        x_next = x
        #print(x_next)
    return x_next
    

In [7]:
C, d = forward_elimination(A,y)

exact_x = back_substitution(C, d)
exact_x

array([-1., -1., -1.])

In [22]:
x, history = gauss_seidel(M,N,x0,y,3)
#print(history)
#print(np.matmul(A,x))
print("error in GS with 3 iterations:\n", np.max(x - exact_x))

error in GS with 3 iterations:
 0.5468789192578554


In [23]:
z = []
z.append( history[1] - aitken_accel(np.zeros(np.shape(history[0])),history[0], history[1]))
z.append( history[2] - aitken_accel(history[0],history[1], history[2]))
z.append( history[3] - aitken_accel(history[1], history[2], history[3]))

#print(z)
print("error in GS with Aitken acceleration with 3 iterations:\n", np.max(z[2]-exact_x))

error in GS with Aitken acceleration with 3 iterations:
 0.022973000292216494


In [21]:
x_check, history = gauss_seidel(M,N,np.zeros((3)),y, 16)
#print(x_check)
print("error in GS with 15 iterations:", np.max(history[15]-exact_x))
print("error in GS with 16 iterations:", np.max(x_check-exact_x))

error in GS with 15 iterations: 0.025450379373500254
error in GS with 16 iterations: 0.019709463667692217
